In [13]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import scipy as sc
from skimage import transform

In [14]:
from os import listdir
from os.path import isfile, join
path1 = 'data/symptoms_new/'
path2 = 'data/nosymptoms_new/'
file_pos = [path1+f for f in listdir(path1) if isfile(join(path1, f)) and f[-4:] == 'jpeg']
file_neg = [path2+f for f in listdir(path2) if isfile(join(path2, f)) and f[-4:] == 'jpeg']

In [15]:
len(file_pos), len(file_neg)

(595, 1468)

In [22]:
import pandas as pd
file1 = pd.DataFrame()
file1['location'] = file_pos
file1['label'] = 1

file2 = pd.DataFrame()
file2['location'] = file_neg
file2['label'] = 0
file = pd.concat([file1, file2], axis=0)
file = pd.

In [23]:
file

,location,label
0,data/symptoms_new/16576_left.jpeg,1
1,data/symptoms_new/14353_right.jpeg,1
2,data/symptoms_new/11623_left.jpeg,1
3,data/symptoms_new/11162_left.jpeg,1
4,data/symptoms_new/12042_right.jpeg,1
5,data/symptoms_new/11183_left.jpeg,1
6,data/symptoms_new/14251_right.jpeg,1
7,data/symptoms_new/14811_right.jpeg,1
8,data/symptoms_new/12839_left.jpeg,1
9,data/symptoms_new/15898_right.jpeg,1
